## Project 1: World Migration, Causes and Associations

###### Group Evelyn Kitagawa: Neha Lala, Jackie Schneider, Himalia Joshi, Monica Wilson, Lea Yamashiro, Kevin Dunn

In [1]:
%%capture
!pip install wbdata
import wbdata
!pip install cufflinks # IF NECESSARY
import cufflinks as cf
cf.go_offline()
!pip install pandas
import pandas as pd
import numpy as np
import statsmodels.api as sm

#### Cleaning Data

In [23]:
indicators = {"NY.GDP.PCAP.CD":"GDP per capita", 
              "SH.DYN.1019": "Death Probability (5-9)",
              "SH.DYN.2024": 'Death Probability (20-24)',
              "SP.DYN.TFRT.IN":"Total Fertility Rate",
              "SP.POP.GROW":"Population Growth Rate",
              "SP.DYN.AMRT.MA":"Male Mortality",
              "SP.DYN.AMRT.FE":"Female Mortality",
              "SP.POP.1564.FE.ZS":"% Adult Female",
              "SP.POP.TOTL.FE.ZS":"% Female", 
              "SM.POP.NETM": "Net Migration", 
              "SP.POP.TOTL": "Total Population"}
data = wbdata.get_dataframe(indicators)

SOURCE=75
esg_indic = wbdata.get_indicator(source=SOURCE)
esg_indic

id                 name
-----------------  ---------------------------------------------------------------------------------------------------------
AG.LND.AGRI.ZS     Agricultural land (% of land area)
AG.LND.FRLS.HA     Tree Cover Loss (hectares)
AG.LND.FRST.ZS     Forest area (% of land area)
AG.PRD.FOOD.XD     Food production index (2014-2016 = 100)
CC.EST             Control of Corruption: Estimate
EG.CFT.ACCS.ZS     Access to clean fuels and technologies for cooking (% of population)
EG.EGY.PRIM.PP.KD  Energy intensity level of primary energy (MJ/$2017 PPP GDP)
EG.ELC.ACCS.ZS     Access to electricity (% of population)
EG.ELC.COAL.ZS     Electricity production from coal sources (% of total)
EG.ELC.RNEW.ZS     Renewable electricity output (% of total electricity output)
EG.FEC.RNEW.ZS     Renewable energy consumption (% of total final energy consumption)
EG.IMP.CONS.ZS     Energy imports, net (% of energy use)
EG.USE.COMM.FO.ZS  Fossil fuel energy consumption (% of total)
EG.USE.P

In [25]:
mig_away = data[data['Net Migration'] < 0]
mig_away['Net Migration'] = mig_away['Net Migration'].abs()
mig_away['Migration per Capita'] = mig_away['Net Migration']/mig_away['Total Population']
mig_away['Migration Rate (%)'] = mig_away['Migration per Capita']*100
mig_away

GDP per capita  Death Probability (5-9)  \
country                     date                                            
Africa Eastern and Southern 2021     1545.613215                15.237635   
                            2020     1355.805923                15.549840   
                            2019     1507.982881                15.878293   
                            2018     1558.307482                16.214853   
                            2017     1625.286236                16.545213   
...                                          ...                      ...   
Zimbabwe                    1964      282.376856                      NaN   
                            1963      277.532515                      NaN   
                            1962      275.966139                      NaN   
                            1961      279.332656                      NaN   
                            1960      276.643363                      NaN   

                                  Death Probability (20-24)  \
country                     date                              
Africa Eastern and Southern 2021                  13.303625   
                            2020                  13.464270   
                            2019                  13.663853   
                            2018                  13.879416   
                            2017                  14.129777   
...                                                     ...   
Zimbabwe                    1964                        NaN   
                            1963                        NaN   
                            1962                        NaN   
                            1961                        NaN   
                            1960                        NaN   

                                  Total Fertility Rate  \
country                     date                         
Africa Eastern and Southern 2021              4.354709   
                            2020              4.416900   
                            2019              4.482898   
                            2018              4.527705   
                            2017              4.570409   
...                                                ...   
Zimbabwe                    1964              7.261000   
                            1963              7.253000   
                            1962              7.255000   
                            1961              7.233000   
                            1960              7.220000   

                                  Population Growth Rate  Male Mortality  \
country                     date                                           
Africa Eastern and Southern 2021                2.607472      332.823489   
                            2020                2.678184      311.485254   
                            2019                2.691134      302.772370   
                            2018                2.688371      309.165595   
                            2017                2.655672      314.493882   
...                                                  ...             ...   
Zimbabwe                    1964                3.119878      358.796000   
                            1963                3.115408      361.354000   
                            1962                3.105320      366.062000   
                            1961                3.094865      369.918000   
                            1960                     NaN      374.000000   

                                  Female Mortality  % Adult Female   % Female  \
country                     date                                                
Africa Eastern and Southern 2021        237.602676       55.774440  50.478916   
                            2020        224.232001       55.549322  50.486544   
                            2019        218.643251       55.348700  50.495993   
                            2018        224.119316       55.174

In [29]:
# getting population dataset 

indicators = {"SP.POP.TOTL": "Total Population", "SM.POP.NETM":"Net Migration"}

data = wbdata.get_dataframe(indicators)

# Make years ints instead of strings
data.reset_index(inplace=True)
data['date'] = data['date'].astype(int)

# index the table by country instead of year 
data.set_index(['country'],inplace=True)
data

,date,Total Population,Net Migration
country,,,
Africa Eastern and Southern,2022,720859132.0,NaN
Africa Eastern and Southern,2021,702977106.0,-179444.0
Africa Eastern and Southern,2020,685112979.0,-48955.0
Africa Eastern and Southern,2019,667242986.0,-187410.0
Africa Eastern and Southern,2018,649757148.0,-366105.0
...,...,...,...
Zimbabwe,1964,4310332.0,-10064.0
Zimbabwe,1963,4177931.0,-9369.0
Zimbabwe,1962,4049778.0,-8931.0


In [21]:
var_labels = {"SM.POP.NETM":"Net Migration"}
world_migration = wbdata.get_dataframe(var_labels)
world_migration_np = world_migration.dropna()
population_statistics = world_migration_np.groupby('country').agg({
    'Net Migration': ['sum', 'mean']
}).reset_index()
world_migration_np = population_statistics[population_statistics['Net Migration']<0]
world_migration_np

ValueError: cannot join with no overlapping index names

In [ ]:
population_statistics = world_migration_np.groupby('country').agg({
    'Net Migration': ['sum', 'mean']
}).reset_index()
population_statistics

In [28]:
mig_away = mig_away.dropna()
population_statistics = mig_away.groupby('country').agg({
    'Migration per Capita': ['sum', 'mean']
}).reset_index()
population_statistics

country Migration per Capita          
                                                  sum      mean
0                    Afghanistan             0.072475  0.007247
1    Africa Eastern and Southern             0.025730  0.000858
2     Africa Western and Central             0.019213  0.000640
3                        Albania             0.424827  0.013276
4                        Algeria             0.029802  0.000961
..                           ...                  ...       ...
192                     Viet Nam             0.017016  0.000608
193           West Bank and Gaza             0.155771  0.006490
194                  Yemen, Rep.             0.075420  0.002357
195                       Zambia             0.046117  0.004192
196                     Zimbabwe             0.219511  0.007569

[197 rows x 3 columns]

In [5]:
data = wbdata.get_dataframe(indicators)

# Make years ints instead of strings
data.reset_index(inplace=True)
data['date'] = data['date'].astype(int)

# index the table by country instead of year 
data.set_index(['country'],inplace=True)

mig_away = data[data['Net Migration'] < 0]
mig_away['Net Migration'] = mig_away['Net Migration'].abs()
mig_away['Migration per Capita'] = mig_away['Net Migration']/mig_away['Total Population']
mig_away['Migration Rate (%)'] = mig_away['Migration per Capita']*100

NameError: name 'indicators' is not defined

#### Population Statistics

In [ ]:
def population(year=1966, sex='Male', age_range=(18,26), place='wrld'):
    print("Migration Rate", Migration Rate (%), "Total Population", SP.POP.TOTL)